### Script to generate FATES parameters using LHS

In [1]:
from scipy.stats import qmc
import numpy as np

import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re

import functions_py.modp as mp


In [2]:
random.seed(32)

#### Read in min and max values for each parameter and pft 

In [3]:
param_ranges_full = pd.read_csv('/global/homes/j/jneedham/DBEN_cbudget_2024/param_files/ensembles/bia/BIA_ensemble_params_trimmed.csv')
param_ranges = param_ranges_full[['param', 'value_min', 'value_max', 'pft', 'organ']]

# number of parameters
n_params = len(param_ranges)

# number of PFTs - some are global so subtract one
n_pfts = len(pd.unique(param_ranges['pft'])) - 1

param_names = list(param_ranges.param)
pfts = list(param_ranges.pft)
organs = list(param_ranges.organ)

print(param_ranges)


                                     param  value_min  value_max  pft  organ
0                         fates_mort_bmort      0.015     0.0200    2    NaN
1                       fates_wood_density      0.600     0.7000    2    NaN
2                    fates_leaf_vcmax25top     46.000    50.0000    2    NaN
3              fates_alloc_storage_cushion      2.400     3.6000    1    NaN
4                    fates_leaf_vcmax25top     52.000    55.0000    1    NaN
5                         fates_mort_bmort      0.010     0.0140    1    NaN
6  fates_maintresp_leaf_atkin2017_baserate      1.000     1.4995    1    NaN
7                        fates_leaf_slatop      0.005     0.0100    1    NaN


In [4]:
n_inst = 256

sampler = qmc.LatinHypercube(d=n_params)
sample = sampler.random(n=n_inst)

# scale to parameter ranges
l_bounds = param_ranges['value_min']
u_bounds = param_ranges['value_max']

scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

print(scaled_sample[0,:])

[1.55015274e-02 6.12453785e-01 4.84151750e+01 3.24822872e+00
 5.28341299e+01 1.37886947e-02 1.49097513e+00 6.13071675e-03]


In [5]:
## Read in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

input_fname = '/global/homes/j/jneedham/DBEN_cbudget_2024/param_files/fates_params_bia_base_api29.nc'

# for each sample 
for i in range(0,n_inst) :
    
    # final parameter file name
    fout = '/global/homes/j/jneedham/DBEN_cbudget_2024/param_files/ensembles/bia/fates_params_bia_ens_{0}.nc'.format(i+1)
    
    shutil.copyfile(input_fname, fout)                                                                                                                             
   
     # loop through each parameter and apply either to the correct pft or globally
    for j in range(0, n_params) : 
        
        var = param_names[j]
        pft = pfts[j]
        organ = organs[j]
        
        val = scaled_sample[i, j]
        
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
        